In [53]:
import numpy as np

movieTitles = open("hw8_movieTitles.txt").read().split() # 50x1 mat of movie titles
movieCount = len(movieTitles)
studentPids = open("hw8_studentPIDs.txt").read().split() # 279x1 mat of student PIDs 
studentCount = len(studentPids)
ratings = [] #279x50 mat of userxmovie ratings

for l in open("hw8_ratings.txt"):
    temp = l.rstrip('\n').split()
    ratings.append(temp)

ratings = np.array(ratings)
probZinit = np.loadtxt("hw8_probZ_init.txt") #1x4 
groupCount = len(probZinit) # the number of groups we are considering
probRgivenZ = np.loadtxt("hw8_probRgivenZ_init.txt") #50x4

4


In [52]:
from collections import Counter
moviePopularityRating = [Counter(ratings[:,t])['1']/(Counter(ratings[:,t])['1']+Counter(ratings[:,t])['0']) for t in range(movieCount)]
for index in np.argsort(moviePopularityRating)[::-1]:
    print(movieTitles[index])

Inception
The_Dark_Knight_Rises
Interstellar
Shutter_Island
The_Martian
The_Social_Network
Now_You_See_Me
12_Years_a_Slave
The_Theory_of_Everything
Gone_Girl
Harry_Potter_and_the_Deathly_Hallows:_Part_2
Toy_Story_3
Black_Swan
Harry_Potter_and_the_Deathly_Hallows:_Part_1
Wolf_of_Wall_Street
The_Avengers
Midnight_in_Paris
Frozen
The_Girls_with_the_Dragon_Tattoo
Django_Unchained
Room
Ex_Machina
Her
The_Revenant
X-Men:_First_Class
The_Great_Gatsby
Jurassic_World
Star_Wars:_The_Force_Awakens
Les_Miserables
Captain_America:_The_First_Avenger
21_Jump_Street
Avengers:_Age_of_Ultron
The_Help
American_Hustle
The_Perks_of_Being_a_Wallflower
Iron_Man_2
The_Hateful_Eight
Fast_Five
The_Hunger_Games
Pitch_Perfect
Thor
Drive
Mad_Max:_Fury_Road
Man_of_Steel
World_War_Z
Bridemaids
Prometheus
Magic_Mike
The_Last_Airbender
Fifty_Shades_of_Grey


In [66]:
probZgivenR = np.zeros(shape=(groupCount, studentCount))

def Estep():
    global probRgivenZ, probZgivenR, probZinit
    for s in range(studentCount):
        sRatings = ratings[s]
        ratedIndices = [i for i, x in enumerate(sRatings) if x == '1' or x == '0']
        probZgivenRt = np.ones(shape=(groupCount))
        denom = 0;
        for g in range(groupCount):
            probZgivenRt[g] = probZinit[g];
            for index in ratedIndices:
                if(sRatings[index]=='1'):
                    probZgivenRt[g] *= probRgivenZ[index][g]
                else:
                    probZgivenRt[g] *= (1-probRgivenZ[index][g])
            denom += probZgivenRt[g]
        for g in range(groupCount):
            probZgivenR[g][s] = probZgivenRt[g]/denom

In [67]:
def Mstep():
    global probRgivenZ, probZgivenR, probZinit
    #updating Zinit
    for g in range(groupCount):
        updatedVal = 0
        for s in range(studentCount):
            updatedVal += probZgivenR[g][s]
        updatedVal = updatedVal/studentCount
    #updating probRgivenZ
    for g in range(groupCount):
        for m in range(movieCount):
            updatedVal = 0
            denom = 0
            for s in range(studentCount):
                if(ratings[s][m]=='1'):
                    updatedVal += probZgivenR[g][s]
                elif(ratings[s][m]=='?'):
                    updatedVal += (probZgivenR[g][s]*probRgivenZ[m][g])
                denom += probZgivenR[g][s]
            probRgivenZ[m][g] = updatedVal/denom;

In [68]:
import math
def logLikelihood():
    global probRgivenZ, probZgivenR, probZinit
    denom = studentCount
    ll = 0;
    for s in range(studentCount):
        llisum = 0
        for g in range(groupCount):
            update = probZinit[g]
            ratedIndices = [i for i,x in enumerate(ratings[s]) if x=='1' or x=='0']
            for index in ratedIndices:
                if(ratings[s][index] == '1'):
                    update *= probRgivenZ[index][g]
                else:
                    update *= (1-probRgivenZ[index][g])
            llisum += update
        ll += math.log(llisum)
    ll = ll/denom
    print(ll)

In [69]:
print(logLikelihood()) #At 0th step

-23.68194303920656
None


In [70]:
for i in range(64):
    Estep()
    Mstep()
    print("Iteration: "+str(i+1))
    print(logLikelihood())

Iteration: 1
-14.399733170699335
None
Iteration: 2
-12.939273661180074
None
Iteration: 3
-12.43497116937426
None
Iteration: 4
-12.199641945804746
None
Iteration: 5
-12.070447450378559
None
Iteration: 6
-11.988088427805586
None
Iteration: 7
-11.929030596075881
None
Iteration: 8
-11.88286391861488
None
Iteration: 9
-11.84478217476614
None
Iteration: 10
-11.81312813557844
None
Iteration: 11
-11.786176160241522
None
Iteration: 12
-11.761551643871815
None
Iteration: 13
-11.739778227624766
None
Iteration: 14
-11.722867422174433
None
Iteration: 15
-11.710393235384176
None
Iteration: 16
-11.700961722279278
None
Iteration: 17
-11.693566233075046
None
Iteration: 18
-11.687620822641579
None
Iteration: 19
-11.68274865577094
None
Iteration: 20
-11.678631683781251
None
Iteration: 21
-11.674963840034065
None
Iteration: 22
-11.671470086192798
None
Iteration: 23
-11.668023951176503
None
Iteration: 24
-11.66474793737328
None
Iteration: 25
-11.661743602564057
None
Iteration: 26
-11.658825839859485
None
I

In [74]:
myIndex = studentPids.index('A53244405') #204 index
unseenIndexes = [i for i, x in enumerate(ratings[myIndex]) if x=='?']
probMyUnseen = np.ones(movieCount)
for u in unseenIndexes:
    print(movieTitles[u])
    unseenProb = 0;
    for g in range(groupCount):
        probTemp = probRgivenZ[u][g]
        probTemp *= probZgivenR[g][myIndex]
        unseenProb += probTemp
    probMyUnseen[u] = unseenProb
    print(unseenProb)

Black_Swan
0.830150240275
Shutter_Island
0.995890568649
The_Last_Airbender
0.357467447096
Bridemaids
0.405782343041
The_Girls_with_the_Dragon_Tattoo
0.69778030076
Drive
0.627701482606
Midnight_in_Paris
0.992185085701
Prometheus
0.336905205665
Django_Unchained
0.914797022493
Les_Miserables
0.762539308097
21_Jump_Street
0.83211804867
Magic_Mike
0.0157587720428
Her
0.857637774237
12_Years_a_Slave
0.930895899606
World_War_Z
0.488338343605
American_Hustle
0.826911565279
Ex_Machina
0.932765098079
Room
0.962861761528
The_Martian
0.929283278119
The_Hateful_Eight
0.487441029611
The_Revenant
0.815816600029
